<a href="https://colab.research.google.com/github/safednn-nasa/Prophecy/blob/master/ACASX_Prophecy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import io
import os
from collections import namedtuple
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import tree
from tqdm import tqdm
import operator
import pandas as pd


In [0]:
!pip3 install -U pybind11

Requirement already up-to-date: pybind11 in /usr/local/lib/python3.6/dist-packages (2.4.3)


In [0]:
LAYER = 5
LABEL = 0

##**READ 384221 ACASX INPUTS WITH KNOWN LABELS**

In [0]:
acas_train = np.empty([384221,5],dtype=float)
acas_train_labels = np.zeros(384221,dtype=int)

def read_inputs_from_file(inputFile):
    global acas_train, acas_train_labels, num
    with open(inputFile) as f:
        lines = f.readlines()
        print(len(lines), "examples")
        acas_train = np.empty([len(lines),5],dtype=float)
        acas_train_labels = np.zeros(len(lines),dtype=int)
        
        for l in range(len(lines)):
            k = [float(stringIn) for stringIn in lines[l].split(',')] #This is to remove the useless 1 at the start of each string. Not sure why that's there.
            #acas_train[l+num] = np.zeros(5,dtype=float) #we're asuming that everything is 2D for now. The 1 is just to keep numpy happy.
            if len(k) > 5:
              lab = int(k[5])
              #if ((lab == 0) or (lab == 2)):
              #  lab = 0
              #else:
              #  lab = 1
              acas_train_labels[l+num] = lab
            
            count = 0
            for i in range(0,5):
                #print(count)
                acas_train[l+num][i] = k[i]
                
                #print(k[i])
            
                
        
              
#url1 = 'https://github.com/hayesconverse/sym_convnn/blob/master/MNIST_ITR_REL/mnist_10_layer.txt' 
!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/clusterinACAS_0_short.csv -O ./clusterinACAS_0_shrt.csv
#!wget https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/COV_prop6.csv -O ./COV_prop6.csv
#!wget https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/COV_prop6_1.csv -O ./COV_prop6_1.csv
#!wget https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/prop1_cov.csv -O ./prop1_cov.csv
#!wget https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/clusterinACAS_0_c2.csv -O ./clusterinACAS_0_c2.csv
#!wget https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/clusterinACAS_0_c3.csv -O ./clusterinACAS_0_c3.csv
#!wget https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/clusterinACAS_0_c4.csv -O ./clusterinACAS_0_c4.csv
#!wget https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/clusterinACAS_0_c5.csv -O ./clusterinACAS_0_c5.csv
#!wget https://raw.githubusercontent.com/hayesconverse/sym_convnn/master/clusterinACAS_0_c6.csv -O ./clusterinACAS_0_c6.csv
num = 0
#read_inputs_from_file('./COV_prop6_1.csv')
#prop6_inputs_part1 = acas_train
#read_inputs_from_file('./COV_prop6.csv')
#prop6_inputs_part2 = acas_train
#read_inputs_from_file('./prop1_cov.csv')
#prop1_inputs = acas_train

read_inputs_from_file('./clusterinACAS_0_shrt.csv')

#num = num + 458066
#read_inputs_from_file('./clusterinACAS_0_c2.csv')
#num = num + 460342
#read_inputs_from_file('./clusterinACAS_0_c3.csv')
#num = num + 462166
#read_inputs_from_file('./clusterinACAS_0_c4.csv')
#num = num + 467893
#read_inputs_from_file('./clusterinACAS_0_c5.csv')
#num = num + 467646
#read_inputs_from_file('./clusterinACAS_0_c6.csv')

print((acas_train).shape)



--2019-12-06 19:21:42--  https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/clusterinACAS_0_short.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15903651 (15M) [text/plain]
Saving to: ‘./clusterinACAS_0_shrt.csv’

./clusterinACAS_0_s 100%[===================>]  15.17M  --.-KB/s    in 0.07s   

2019-12-06 19:21:42 (208 MB/s) - ‘./clusterinACAS_0_shrt.csv’ saved [15903651/15903651]

384221 examples
(384221, 5)


##**DEFINE THE ACASX MODEL IN TENSORFLOW 6 LAYERS WITH 50 HIDDEN NODES, 5 INPUTS AND 5 OUTPUTS**

In [0]:

def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=name)

def bias_variable(shape, name):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name=name)

def fc2d(x, W):
  return tf.nn.fc2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def create_model():
    x = tf.identity(tf.placeholder(tf.float32, shape=[None, 5]), name="input")
    y_ = tf.placeholder(tf.float32, shape=[None, 5])
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    #LAYER 0
    W_fc1 = weight_variable([5, 50],name='w_fc1')
    b_fc1 = bias_variable([50],name='b_fc1')
    x_image = tf.reshape(x, [-1, 5])
    h_fc1 = tf.nn.relu(tf.matmul(x_image, W_fc1) + b_fc1)
    h_fc1_iden = tf.identity(h_fc1,name='h_fc1')
    
    #LAYER 1
    W_fc2 = weight_variable([50, 50],name='w_fc2')
    b_fc2 = bias_variable([50],name='b_fc2')
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
    h_fc2_iden = tf.identity(h_fc2,name='h_fc2')
    
    #LAYER 2
    W_fc3 = weight_variable([50, 50],name='w_fc3')
    b_fc3 = bias_variable([50],name='b_fc3')
    h_fc3 = tf.nn.relu(tf.matmul(h_fc2, W_fc3) + b_fc3)
    h_fc3_iden = tf.identity(h_fc3,name='h_fc3')
    
    #LAYER 3
    W_fc4 = weight_variable([50, 50],name='w_fc4')
    b_fc4 = bias_variable([50],name='b_fc4')
    h_fc4 = tf.nn.relu(tf.matmul(h_fc3, W_fc4) + b_fc4)
    h_fc4_iden = tf.identity(h_fc4,name='h_fc4')
    
    #LAYER 4
    W_fc5 = weight_variable([50, 50],name='w_fc5')
    b_fc5 = bias_variable([50],name='b_fc5')
    h_fc5 = tf.nn.relu(tf.matmul(h_fc4, W_fc5) + b_fc5)
    h_fc5_iden = tf.identity(h_fc5,name='h_fc5')
    
    #LAYER 5
    W_fc6 = weight_variable([50, 50],name='w_fc6')
    b_fc6 = bias_variable([50],name='b_fc6')
    h_fc6 = tf.nn.relu(tf.matmul(h_fc5, W_fc6) + b_fc6)
    h_fc6_iden = tf.identity(h_fc6,name='h_fc6')

    #LAYER 6
    W_fc7 = weight_variable([50, 5],name='w_fc7')
    b_fc7 = bias_variable([5],name='b_fc7')
   
  
    y_fc = tf.matmul(h_fc6, W_fc7) + b_fc7
    h_y_fc_iden = tf.identity(y_fc,name='y_fc')
    
    
    prediction = tf.identity(tf.nn.softmax(y_fc), name="import/prediction")
    
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_fc))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmin(y_fc, 1), tf.argmax(y_, 1))
    
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    return cross_entropy, accuracy, x, keep_prob, y_fc, y_, W_fc1, W_fc2, W_fc3, W_fc4, W_fc5, W_fc6, W_fc7, b_fc1, b_fc2, b_fc3, b_fc4, b_fc5, b_fc6, b_fc7, h_fc1, h_fc2, h_fc3, h_fc4, h_fc5, h_fc6
  

### **Restore a pretrained ACASX model FROM A .NN OR TXT FILE**

In [0]:
def read_weights_from_file(inputFile):
    global weightMatrix, biasMatrix
    weightMatrix = np.empty(7, dtype=list)
    biasMatrix = np.empty(7, dtype=list)
    
    weightMatrix_layer0 = np.zeros([5,50], dtype=float)
    biasMatrix_layer0 = np.zeros([50], dtype=float)
    weightMatrix_layer1 = np.zeros([50,50], dtype=float)
    biasMatrix_layer1 = np.zeros([50], dtype=float)
    weightMatrix_layer2 = np.zeros([50,50], dtype=float)
    biasMatrix_layer2 = np.zeros([50], dtype=float)
    weightMatrix_layer3 = np.zeros([50,50], dtype=float)
    biasMatrix_layer3 = np.zeros([50], dtype=float)
    weightMatrix_layer4 = np.zeros([50,50], dtype=float)
    biasMatrix_layer4 = np.zeros([50], dtype=float)
    weightMatrix_layer5 = np.zeros([50,50], dtype=float)
    biasMatrix_layer5 = np.zeros([50], dtype=float)
    weightMatrix_layer6 = np.zeros([50,5], dtype=float)
    biasMatrix_layer6 = np.zeros([5], dtype=float)
    
    with open(inputFile) as f:
        lines = f.readlines()
        currentLine = 0
        prevlayer = -1
        for indx in range(0,len(lines)):
            vals = lines[currentLine].split(',')
            #print vals
            layer = int(vals[0])
            isBias = int(vals[1])
            OutNum = int(vals[2])
            InNum = int(vals[3]) - 1
            
         
            if ((prevlayer == -1) or (prevlayer != layer)):
              if (prevlayer == 0):
                  weightMatrix[prevlayer] = weightMatrix_layer0
                  biasMatrix[prevlayer] = biasMatrix_layer0
              if (prevlayer == 1):
                  weightMatrix[prevlayer] = weightMatrix_layer1
                  biasMatrix[prevlayer] = biasMatrix_layer1
              if (prevlayer == 2):
                  weightMatrix[prevlayer] = weightMatrix_layer2
                  biasMatrix[prevlayer] = biasMatrix_layer2
              if (prevlayer == 3):
                  weightMatrix[prevlayer] = weightMatrix_layer3
                  biasMatrix[prevlayer] = biasMatrix_layer3
              if (prevlayer == 4):
                  weightMatrix[prevlayer] = weightMatrix_layer4
                  biasMatrix[prevlayer] = biasMatrix_layer4
              if (prevlayer == 5):
                  weightMatrix[prevlayer] = weightMatrix_layer5
                  biasMatrix[prevlayer] = biasMatrix_layer5
              prevlayer = layer
              
             
              
            if (isBias == 0):
                if (layer == 0):
                  weightMatrix_layer0[InNum][OutNum] = float(vals[4])
                if (layer == 1):
                  weightMatrix_layer1[InNum][OutNum] = float(vals[4])
                if (layer == 2):
                  weightMatrix_layer2[InNum][OutNum] = float(vals[4])
                if (layer == 3):
                  weightMatrix_layer3[InNum][OutNum] = float(vals[4])   
                if (layer == 4):
                  weightMatrix_layer4[InNum][OutNum] = float(vals[4])
                if (layer == 5):
                  weightMatrix_layer5[InNum][OutNum] = float(vals[4])
                if (layer == 6):
                  weightMatrix_layer6[InNum][OutNum] = float(vals[4])   
               
            else:
                if (layer == 0):
                  biasMatrix_layer0[OutNum] = float(vals[4])
                if (layer == 1):
                  biasMatrix_layer1[OutNum] = float(vals[4])
                if (layer == 2):
                  biasMatrix_layer2[OutNum] = float(vals[4])
                if (layer == 3):
                  biasMatrix_layer3[OutNum] = float(vals[4])   
                if (layer == 4):
                  biasMatrix_layer4[OutNum] = float(vals[4])
                if (layer == 5):
                  biasMatrix_layer5[OutNum] = float(vals[4])
                if (layer == 6):
                  biasMatrix_layer6[OutNum] = float(vals[4])   
               
                
            currentLine += 1
            
    
    weightMatrix[6] = weightMatrix_layer6
    biasMatrix[6] = biasMatrix_layer6
    
    for indx in range(0,7):
          print(indx,weightMatrix[indx])
          print(indx,biasMatrix[indx])
              
!wget 'https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/ACASX_layer.txt' -O ./ACASX_layer.txt

read_weights_from_file('./ACASX_layer.txt')

tf.reset_default_graph()
sess = tf.InteractiveSession()
cross_entropy, accuracy, x, keep_prob, y_fc, y_, W_fc1, W_fc2, W_fc3, W_fc4, W_fc5, W_fc6, W_fc7, b_fc1, b_fc2, b_fc3, b_fc4, b_fc5, b_fc6, b_fc7, h_fc1, h_fc2, h_fc3, h_fc4, h_fc5, h_fc6 = create_model()
feed_dict = {x:acas_train, y_: np.eye(5)[acas_train_labels], keep_prob: 1.0, W_fc1: weightMatrix[0], b_fc1: biasMatrix[0], W_fc2: weightMatrix[1], b_fc2: biasMatrix[1], W_fc3: weightMatrix[2], b_fc3: biasMatrix[2], W_fc4: weightMatrix[3], b_fc4: biasMatrix[3], W_fc5: weightMatrix[4], b_fc5: biasMatrix[4], W_fc6: weightMatrix[5], b_fc6: biasMatrix[5], W_fc7: weightMatrix[6],  b_fc7:biasMatrix[6]}

test_accuracy = accuracy.eval(feed_dict)
print("Test accuracy %g"%(test_accuracy))



--2019-12-06 19:21:46--  https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/ACASX_layer.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264199 (258K) [text/plain]
Saving to: ‘./ACASX_layer.txt’

./ACASX_layer.txt   100%[===================>] 258.01K  --.-KB/s    in 0.02s   

2019-12-06 19:21:46 (16.0 MB/s) - ‘./ACASX_layer.txt’ saved [264199/264199]

0 [[ 5.40060e-02 -1.12374e+00  1.96019e-01 -1.63015e+00 -3.55133e-01
  -3.41920e-02  4.02002e-01 -1.41575e+00 -1.44498e+00 -1.12740e+00
   6.13717e-01  9.33699e-01 -4.08030e-02 -2.12265e-01  3.20180e-02
  -1.24050e-02 -1.22430e-01  6.01740e-02  8.33030e-02 -1.18804e+00
  -1.03537e-01 -8.53900e-02 -1.93840e-02  1.66870e-02  1.67600e-03
  -8.97525e-01 -9.33220e-01  1.60333e-01  8.34020e-02  2.84019e

In [0]:
if (LAYER == 1):
  curr_lay = h_fc1
  prev_lay = h_fc1
 
if (LAYER == 2):
  curr_lay = h_fc2
  prev_lay = h_fc1
  
if (LAYER == 3):
  curr_lay = h_fc3
  prev_lay = h_fc2
 
if (LAYER == 4):
  curr_lay = h_fc4
  prev_lay = h_fc3

if (LAYER == 5):
  curr_lay = h_fc5
  prev_lay = h_fc4

if (LAYER == 6):
  curr_lay = h_fc6
  prev_lay = h_fc5

In [0]:
import copy

def get_prediction(inps, tensor=y_fc, batch_size=100):
  def get_prediction_batch(batch):
    #feed = {x: np.array(batch), keep_prob:1.0}
    feed = {x: np.array(batch), keep_prob:1.0, W_fc1: weightMatrix[0], b_fc1: biasMatrix[0], W_fc2: weightMatrix[1], b_fc2: biasMatrix[1], W_fc3: weightMatrix[2], b_fc3: biasMatrix[2], W_fc4: weightMatrix[3], b_fc4: biasMatrix[3], W_fc5: weightMatrix[4], b_fc5: biasMatrix[4], W_fc6: weightMatrix[5], b_fc6: biasMatrix[5],W_fc7: weightMatrix[6], b_fc7: biasMatrix[6]}
    return sess.run(tensor, feed_dict=feed)
  n = len(inps)
  if n%batch_size == 0:
    batches = [inps[i*batch_size:(i+1)*batch_size] for i in range(int(n/batch_size))]
  else:
    batches = [inps[i*batch_size:(i+1)*batch_size] for i in range(int(n/batch_size) +1)]    
  batch_predictions = [get_prediction_batch(b) for b in tqdm(batches)]
  return np.concatenate(tuple(batch_predictions), axis=0)


## Extracting Invariant Candidates

In [0]:
def fingerprint_suffix(inps,ten):
  # Below t_fc1 is the final fully connected layer of size 1024.
  return (get_prediction(inps, tensor=ten)>0.0).astype('int')

def fingerprint_signature(inps,ten = h_fc5):
  # Below t_fc1 is the final fully connected layer of size 1024.
  return (get_prediction(inps, tensor=ten)>0.0).astype('int')


In [0]:
# train_suffixes, train_predictions are in the same order
# as mnist.train.images. Henceforth when we use the index i we will
# be referring to mnist.train.images[i].
train_suffixes = fingerprint_suffix(acas_train,curr_lay)
print("Suffixes computed for all training data")
train_predictions = np.argmin(get_prediction(acas_train), axis=1)

#ACASX accuracy is about 76.6%

  2%|▏         | 77/3843 [00:00<00:04, 769.18it/s]

Suffixes computed for all training data


100%|██████████| 3843/3843 [00:04<00:00, 811.54it/s]


In [0]:
def describe_input(i, training=True):
  print("Input", i)
  print("SHOW INPUT", acas_train[i])
  print("Groundtruth", acas_train_labels[i])
  print("Prediction", train_predictions[i])

### Build the Decision Tree

In [0]:
# Basic decision tree
basic_estimator = tree.DecisionTreeClassifier()
basic_estimator.fit(train_suffixes, train_predictions)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

### Examine clusters/invariants

In [0]:
def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(neuron_ids, neuron_sig,suffixes=train_suffixes):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

def is_consistent_cluster(cluster, predictions):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = predictions[cluster[0]]
  for i in cluster:
    if predictions[i] != pred:
      return False
  return True

def is_misclassified(i):
  return train_predictions[i] != acas_train_labels[i]

def visualize_conductances(img, label, neuron_ids, only_on=False):
  # Visualize the conductances for the provided image.
  # Args:
  # - img: the provided mnist image
  # - label: prediction label w.r.t. conductance must be computed
  # - neuron_ids: list of neurons indices from the suffix tensor for which
  #    conductances must be computed.
  # - only_on: If True then conductance is computed only for those neurons
  #    that are on for the given image. 
  vis = [mnist_to_pil_img(img)]
  suffix = fingerprint_suffix([img])
  sumigc = 0.0
  for i, id in enumerate(neuron_ids):
    if only_on and suffix[i] != 1:
      continue  
    igc = conductance(img, label, neuron_id=id)
    # igc = conductances[id]
    sumigc = sumigc + igc
  
  
  avgigc = sumigc / len(neuron_ids)
  maxval = abs(max(avgigc, key=abs))
  minval = abs(min(avgigc, key=abs))
  threshold = (maxval - minval)/2.0
  print("MAX ATR:", maxval, "MIN ATR:", minval, "THRESH:", threshold)
  avgigc = 1.0 * avgigc * (abs(avgigc) >= threshold)
  
  
  vis.append(visualize_attrs2(255*mnist_to_rgb(img), mnist_to_rgb(avgigc)))
  return combine(vis)

def get_invariant_inp(estimator, ref_id, suffixes):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - inp: reference input, shape <784,>
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist_inp_images[ref_id]
  ref_suffix = suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,suffixes)
  imgs = []
  cnt = 0
  for indx1 in range(0,len(cluster)):
    img = mnist.train.images(cluster[indx1])
    fnd = 1
    for i in range(0,len(img)):
      if (ref_img[i] != img[i]):
        fnd = 0
        break
    if (fnd == 1):
        ref_id = cnt
    cnt = cnt + 1
    imgs.append(img)
    
  imgs_suffixes = fingerprint_signature(imgs,t_fc2)
  ref_suffix = imgs_suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,imgs_suffixes)
    
  return cluster, neuron_ids, neuron_sig

def get_invariant(estimator, ref_id):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into mnist.train.images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist.train.images[ref_id]
  ref_suffix = train_suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  neuron_sig = ref_suffix[neuron_ids]
  cluster = get_suffix_cluster(neuron_ids, neuron_sig)
  return cluster, neuron_ids, neuron_sig


def get_all_invariants(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]
  
  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
    paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print("Obtained all paths")
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants


def describe_cluster(cluster, neuron_ids, show_samples=False):
  neuron_sig = train_suffixes[cluster[0]][neuron_ids]
  print("Num neurons in invariant", len(neuron_ids))
  print("Neuron id and signature")
  
  for i in range(0,len(neuron_ids)):
    print("id:", neuron_ids[i], "sig:", neuron_sig[i])
  
  print("Cluster size: ", len(cluster))
  print("Num misclassified", len([i for i in cluster if is_misclassified(i)]))
  if show_samples:
    for i in range(10):
      images = []
      for j in range(10):
        if 10*i + j >= len(cluster):
          break
        images.append(mnist_to_pil_img(mnist.train.images[cluster[10*i+j]]))
      if len(images) > 0:
        show_img(combine(images))
  

def describe_invariants_all_labels(all_invariants,prevlayer = h_fc1,layer = h_fc2,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  
  print("PRINTING PURE RULES WITH SUPPORT MORE THAN 50 FOR EVERY LABEL:");
  for cl, invs in all_invariants.items():
    if (cl == -1):
      continue
    
    for indx in range (0, len(invs)):
      inv = invs[indx]
      cls = get_suffix_cluster(inv[0],inv[1],suffixes)
      
      neurons = inv[0]
      signature = inv[1]

      if (len(cls) <= 10000):
        continue
      print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"), Support:",inv[2],", Num misclassified", len([i for i in cls if is_misclassified(i)]));

#      invoke_marabou_chk(LAYER,neurons,signature,cl)

      if (COMMON == True):
          common_nodes(cls,suffixes)

      if (DEC_PREFX == True):
          decision_prefs(cls,suffixes)

  return
  
def common_nodes(cls,suffixes):
    cnt = 0
    common = np.zeros(10,dtype=int)
    prev = np.zeros(10,dtype=int)
    
    for indx in range(0, len(cls)):
        i = cls[indx]
        cnt = cnt + 1
        for j in range(0,len(suffixes[i])):
          if (common[j] == -1):
             continue
          if ((indx != 0) and (suffixes[i][j] != prev[j])):
             common[j] = -1
          else:
             common[j] = suffixes[i][j]
          prev[j] = suffixes[i][j]


    print('COMMON NODES IN CLUSTER for CLASS:',cl,cnt)
    com = []
    for k in range(0,len(common)):
        if (common[k] != -1):
           com.append((k,common[k]))
    print(com)

    return
    
def decision_prefs(cls,suffixes):
    images = mnist.train.images
    imgsCom = []
    imgs = []
    for indx in range(0, len(cls)):
        print('IMG:')
        print(list(zip(images[cls[indx]])))
        imgs.append(images[cls[indx]])
        imgsCom.append(images[cls[indx]])
            
    dec_prefixes= fingerprint_signature(imgs,layer)
    prefixes = []
    for indx in range(0,len(dec_prefixes)):
       dec_pref = dec_prefixes[indx]
    
       match = 0
       for indx1 in range(0, len(prefixes)):
          match = 1
          for i in range(0,len(prefixes[indx1])):
             if (dec_pref[i] != prefixes[indx1][i]):
                match = 0
                break
          if (match == 1):
             break
    
       if (match == 0):
          prefixes.append(dec_pref)
    
    print('DECISION PREFIXES IN CLUSTER for CLASS:',cl,cnt)
    for k in range(0,len(prefixes)):
      print(prefixes[k])

    return
    
  
  #print('LAYER INPS:')
  #min = np.zeros(10)
  #max = np.zeros(10)
  #for dim in range(0,10):
  #    min[dim] = 1000
  #    max[dim] = -1000
          
  #prevlayer_vals = get_prediction(imgsCom,prevlayer)      
  #print('MIN, MAX LAYER INPS:',len(prevlayer_vals))
  #for i in range(0,len(prevlayer_vals)):
  #    if (i == 0):
  #      print(zip(prevlayer_vals[i]))
  #    for dim in range(0,10):
  #        if ( prevlayer_vals[i][dim] < min[dim]):
  #            min[dim] = prevlayer_vals[i][dim]
  #        if ( prevlayer_vals[i][dim] > max[dim]):
  #            max[dim] = prevlayer_vals[i][dim]
    
  #print('MIN')
  #print(zip(min))
  #print('MAX')
  #print(zip(max))    
    
  #df = pd.DataFrame(df, columns=['Prediction Class', 'Num Instances', 'Num Invariants', 'Num Invariants with cluster size >= 10', 'Size of largest invariant cluster'])
  #df = pd.DataFrame(df,columns=['Pred Class','Total #Neurons','# Invariants'])
  #return df


def describe_all_invariants(all_invariants):
  df = []
  for cl, invs in all_invariants.iteritems(): 
    inv = invs[0]
    clus = get_suffix_cluster(inv[0],inv[1])
    #print(len(clus))
    misCl = 0
    for i in range(0,len(clus)):
      indx = clus[i]
      if (is_misclassified(indx) == True):
        misCl = misCl + 1
    print('class:',cl,',masSup:',inv[2],',#misCl:',misCl)
          

## Property Check

In [0]:
def property_membership(pr):
  inputsProp = []
  for i in range(0,len(acas_train)):
      inp = acas_train[i]
      if ((pr == 6) and ((inp[0] < -0.12929) or (inp[0] > 0.700435))):#p6
          continue
      if ((pr == 10) and ((inp[0] < 0.268978) or (inp[0] > 0.679858))):#p10
          continue
      if ((pr == 9) and ((inp[0] < -0.29523) or (inp[0] > -0.21226))):#p9
          continue
      if ((pr == 5) and ((inp[0] < -0.32427) or (inp[0] > -0.32179))):#p5
          continue
      if ((pr == 8) and ((inp[0] < -0.32842) or (inp[0] > 0.679858))):#p8
          continue
      if ((pr == 7) and ((inp[0] < -0.32842) or (inp[0] > 0.679858))):#p7
          continue
      if (((pr == 2) or (pr == 1)) and ((inp[0] < 0.6) or (inp[0] > 0.679858))):#p2, p1
          continue
      #if ((inp[0] < 0.6) or (inp[0] > 0.679858) ):#p2a
      if ((pr == 4) and ((inp[0] < -0.30353) or (inp[0] > -0.29855))):#p4
          continue

      
      if ((pr == 6) and ((inp[1] < -0.5) or (inp[1] > -0.11141))):#p6
          continue
      if ((pr == 10) and ((inp[1] < 0.111408) or (inp[1] > 0.499999))):#p10
          continue
      if ((pr == 9) and ((inp[1] < -0.06366) or (inp[1] > -0.02228))):#p9
          continue
      if ((pr == 5) and ((inp[1] < 0.031831) or (inp[1] > 0.063662))):#p5
          continue
      if ((pr == 8) and ((inp[1] < -0.5) or (inp[1] > -0.375))):#p8
          continue
      if ((pr == 7) and ((inp[1] < -0.5) or (inp[1] > 0.499999))):#p7
          continue
      if (((pr == 2) or (pr == 1)) and ((inp[1] < -0.5) or (inp[1] > 0.5 ))):#p2, p1
          continue
      #if ((inp[0] < 0.6) or (inp[0] > 0.679858) ):#p2a
      if ((pr == 4) and ((inp[1] < -0.00955) or (inp[1] > 0.009549 ))):#p4
          continue


      if ((pr == 6) and ((inp[2] < -0.5) or (inp[2] > -0.4992))):#p6
          continue
      if ((pr == 10) and ((inp[2] < -0.5) or (inp[2] > -0.49841))):#p10
          continue
      if ((pr == 9) and ((inp[2] < -0.5) or (inp[2] > -0.49841))):#p9
          continue
      if ((pr == 5) and ((inp[2] < -0.5) or (inp[2] > -0.4992) )):#p5
          continue
      if ((pr == 8) and ((inp[2] < -0.01592) or (inp[2] > 0.015915))):#p8
          continue
      if ((pr == 7) and ((inp[2] < -0.5) or (inp[2] > 0.499999) )):#p7
          continue
      if (((pr == 2) or (pr == 1)) and ((inp[2] < -0.5) or (inp[2] > 0.5) )):#p2, p1
          continue
      #if ((inp[0] < 0.6) or (inp[0] > 0.679858) ):#p2a
      if ((pr == 4) and ((inp[2] < 0.493379))):#p4
          continue


      if ((pr == 6) and ((inp[3] < -0.5) or (inp[3] > 0.5))):#p6
          continue
      if ((pr == 10) and ((inp[3] < 0.227273) or (inp[3] > 0.5))):#p10
          continue
      if ((pr == 9) and ((inp[3] < -0.5) or (inp[3] > -0.45455))):#p9
          continue
      if ((pr == 5) and ((inp[3] < -0.5) or (inp[3] > -0.22727) )):#p5
          continue
      if ((pr == 8) and ((inp[3] < -0.045) or (inp[3] > 0.5))):#p8
          continue
      if ((pr == 7) and ((inp[3] < -0.5) or (inp[3] > 0.5))):#p7
          continue
      if (((pr == 2) or (pr == 1)) and ((inp[3] < 0.45) or (inp[3] > 0.5))):#p2, p1
          continue
      #if ((inp[0] < 0.6) or (inp[0] > 0.679858) ):#p2a
      if ((pr == 4) and (inp[3] < 0.3 )):#p4
          continue

      if ((pr == 6) and ((inp[4] < -0.5) or (inp[4] > 0.5))):#p6
          continue
      if ((pr == 10) and ((inp[4] < 0) or (inp[4] > 0.5) )):#p10
          continue
      if ((pr == 9) and ((inp[4] < -0.5) or (inp[4] > -0.375))):#p9
          continue
      if ((pr == 5) and ((inp[4] < -0.5) or (inp[4] > -0.16667))):#p5
          continue
      if ((pr == 8) and ((inp[4] < 0.0) or (inp[4] > 0.5))):#p8
          continue
      if ((pr == 7) and ((inp[4] < -0.5) or (inp[4] > 0.5) )):#p7
          continue
      if (((pr == 2) or (pr == 1)) and ((inp[4] < -0.5) or (inp[4] > -0.45))):#p2, p1
          continue
      #if ((inp[0] < 0.6) or (inp[0] > 0.679858) ):#p2a
      if ((pr == 4) and (inp[4] < 0.3 )):#p4
          continue

      inputsProp.append(i)

  return inputsProp


In [0]:
def property_chk(pr,label,all_invariants,prop_minIn,prop_maxIn,lay=LAYER,suffixes=train_suffixes):
  
  inputsProp = property_membership(pr)
  print("Property:" + str(pr), ",A => LABEL" + str(label))
  print('INPUTS WITHIN A:', len(inputsProp))

  df = []
  
  images = acas_train
  imgsCom = []
  tot = 0
  notCovered = inputsProp
  notCov = []
  for cl, invs in all_invariants.items():
    
    if (cl != label):
      continue
    print(cl, len(invs))
   
    cnt = 0
    
    for invariant in invs:
        cls = get_suffix_cluster(invariant[0],invariant[1],suffixes)
        
        lst3 = [value for value in inputsProp if value in cls]
        withinAcnt = len(lst3)
        notCovered = list(set(notCovered) - set(lst3))
       
        if (withinAcnt > 100):
            print('INVARIANT > 100:' , invariant[0], invariant[1])
            print('SUPPORT > 100:' , invariant[2], ",", len(cls),', A SUPPORT:', withinAcnt)
            
            minI = np.zeros(5)
            maxI = np.zeros(5)
            for ind in range(0,5):
                minI[ind] = 1000
                maxI[ind] = -1000
            
            imgs = []
            print('COVERED:')
            for indx in range(0,len(lst3)):
                index = lst3[indx]
                img = acas_train[index]
                imgs.append(img)
                #print(list(img))
                for ind in range(0,5):
                  if (img[ind] < minI[ind]):
                      minI[ind] = img[ind]
                  if (img[ind] > maxI[ind]):
                      maxI[ind] = img[ind] 
            
            print('INP MIN,MAX covered:')
            print(list(minI))
            print(list(maxI))
      
            if (len(notCovered) > 0):
              for ind in range(0,5):
                minI[ind] = 1000
                maxI[ind] = -1000
              print('NOT COVERED:')
              for indx in range(0,len(notCovered)):
                index = notCovered[indx]
                img = acas_train[index]
                #print(list(img))
              for ind in range(0,5):
                if (img[ind] < minI[ind]):
                  minI[ind] = img[ind]
                if (img[ind] > maxI[ind]):
                  maxI[ind] = img[ind] 
              print('INP MIN,MAX not covered:')
              print(list(minI))
              print(list(maxI))


            # Get the min,max before the layer
            minIn =[]
            maxIn = []
            for ind in range(0,50):
               minIn.append(1000)
               maxIn.append(-1000)

            layer_vals = get_prediction(imgs,h_fc5)      
            print('MIN, MAX LAYER INPS:',len(layer_vals))
            for i in range(0,len(layer_vals)):
                  for dim in range(0,50):
                           if ( layer_vals[i][dim] < minIn[dim]):
                                         minIn[dim] = layer_vals[i][dim]
                           if ( layer_vals[i][dim] > maxIn[dim]):
                                         maxIn[dim] = layer_vals[i][dim]
    
            print('INP MIN,MAX PREV LAYER:')
            print(list(minIn))
            print(list(maxIn))
  
            
            sig = []
            for dim in range(0,50):
                if (layer_vals[0][dim] == 0):
                  sig.append(0)
                else:
                  sig.append(1)
                

            for i in range(1,len(layer_vals)):
                #print(layer_vals[i][0],layer_vals[i][47],layer_vals[i][48] )
                for dim in range(0,50):
                  if (sig[dim] == -1):
                    continue
                  if ((layer_vals[i][dim] > 0) and (sig[dim] == 0)):
                      sig[dim] = -1
                      continue
                  if ((layer_vals[i][dim] == 0) and (sig[dim] == 1)):
                      sig[dim] = -1
                      continue

            print("COMMON SIGNATURE:")
            for dim in range(0,50):
              if (sig[dim] != -1):
                  print("index:" + str(dim) + "=" + str(sig[dim]))
            
            neurons = invariant[0]
            signature = invariant[1]
            
            prov = False
            print(" CHECK I /\ covered_consts on short network => B") ## UPDATE notCov
            prov = invoke_marabou_chk(lay,neurons,signature,label,minIn,maxIn,sig) ## I => LABEL
 
            if (prov == True):
                
                notCovStrs = []
                notCovStrs.append(notCov)
                for i in range(0,len(neurons)):
                  strChk = "ws_"+ str(lay) + "_" + str(neurons[i])
                  if (strChk in notCovStrs):
                    notCov.remove(notCovstr)
                
                print(" CHECK A  => I ") ## COLLECT NOT COVERED CONSTRAINTS
                notCov.append(invoke_marabou_chk_2(lay,neurons,signature,prop_minIn,prop_maxIn)) 

            if (len(notCov) == 0):       
              break

    

In [0]:
!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/marabou_DnC_InternalNodes.elf -O ./marabou_DnC_InternalNodes.elf
!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/ACASXU_run2a_1_1_batch_2000.nnet -O ./ACASXU_run2a_1_1_batch_2000.nnet
!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/ACASXU_run2a_1_2_batch_fc5.nnet -O ./ACASXU_run2a_1_2_batch_fc5.nnet
!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/ACASXU_run2a_1_2_batch_fc5OP.nnet -O ./ACASXU_run2a_1_2_batch_fc5OP.nnet
!pwd
!ls -lt ./marabou_DnC_InternalNodes.elf
!chmod 777 ./marabou_DnC_InternalNodes.elf
!ls -lt ./marabou_DnC_InternalNodes.elf

--2019-12-06 19:22:05--  https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/marabou_DnC_InternalNodes.elf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14808728 (14M) [application/octet-stream]
Saving to: ‘./marabou_DnC_InternalNodes.elf’

./marabou_DnC_Inter 100%[===================>]  14.12M  --.-KB/s    in 0.07s   

2019-12-06 19:22:06 (212 MB/s) - ‘./marabou_DnC_InternalNodes.elf’ saved [14808728/14808728]

--2019-12-06 19:22:07--  https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/ACASXU_run2a_1_1_batch_2000.nnet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connec

In [0]:
## I => B
def invoke_marabou_chk(layer,neurons,signature,label,inp_min = [],inp_max = [], com_sig = [], notCov = []):
  #layer = 1
  #neurons = [4, 8, 7, 1, 0, 2, 5, 3, 9, 6] 
  #signature = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #label = 6
 propsig = True
 not_done = True
 comsig = False
 while( (not_done == True) and (comsig == False)):
  
  for lab_indx in range(0,5):
    if (lab_indx == label):
      continue

    not_done = False
    strInp = ""
    if (len(inp_min) > 0):
      for i in range(0,50):
          strInp = strInp + "x"+ str(i) + " >= " + str(inp_min[i]) + "\n"
          strInp = strInp + "x"+ str(i) + " <= " + str(inp_max[i]) + "\n"
    #print(strInp)

    strInternal = ""
    if (propsig == True):
      for i in range(0,len(neurons)):
        strInternal = strInternal + "ws_"+ str(1) + "_" + str(neurons[i])
        if (signature[i] == 0):
           strInternal = strInternal + " <= 0.0" + "\n"
        else:
           strInternal = strInternal + " >= 0.0"  + "\n"
      propsig = False
    else:
      for dim in range(0,len(com_sig)):
        if (com_sig[dim] == -1):
          continue
        strInternal = strInternal + "ws_"+ str(1) + "_" + str(dim)
        if (com_sig[dim] == 0):
           strInternal = strInternal + " <= 0.0" + "\n"
        else:
           strInternal = strInternal + " >= 0.0"  + "\n"
      comsig = True

    strOP = "+y"+ str(lab_indx) + " -y" + str(label) + " <= -0.001" + "\n"

    #Write to a property file
    file1 = open('property.txt',"w")
    file1.writelines(strInp) 
    #file1.writelines(strInternal) 
    file1.writelines(strOP) 
    file1.close() 

    file1 = open('property.txt',"r")  
    print("PROPERTY FILE IS ")
    print(file1.read())
    file1.close()

    #!./marabou_DnC_InternalNodes.elf ./ACASXU_run2a_1_1_batch_2000.nnet ./property.txt --dnc --num-workers=4 --summary-file=summary1.txt
    !./marabou_DnC_InternalNodes.elf ./ACASXU_run2a_1_2_batch_fc5OP.nnet ./property.txt --summary-file=summary1.txt
    
    print("SUMMARY:")
    f = open('summary1.txt', 'r')
    file_contents = f.read()
    print (file_contents)
    f.close()
    if (file_contents.find('UNSAT') == -1):
        not_done = True
        break
       
 if (not_done == False):
   print("Property proved!")
   return True
 else:
   print ("PROPERTY COULD NOT BE PROVED:")
   return False
  
  #f.close()


In [0]:
# A => I
def invoke_marabou_chk_2(layer,neurons,signature,inp_min,inp_max):
 
    strInp = ""
    for i in range(0,5):
          strInp = strInp + "x"+ str(i) + " >= " + str(inp_min[i]) + "\n"
          strInp = strInp + "x"+ str(i) + " <= " + str(inp_max[i]) + "\n"
    #print(strInp)

    strInternals = []
    for i in range(0,len(neurons)):
        strInternal = "ws_"+ str(layer) + "_" + str(neurons[i])
        if (signature[i] == 0):
           strInternal = strInternal + " >= 0.0" + "\n"
        else:
           strInternal = strInternal + " <= 0.0"  + "\n"
        strInternals.append(strInternal)

    notCov = []
    for strInternal in strInternals:
      #Write to a property file
      file1 = open('property.txt',"w")
      file1.writelines(strInp) 
      file1.writelines(strInternal) 
      file1.close() 

      file1 = open('property.txt',"r")  
      print("PROPERTY FILE IS ")
      print(file1.read())
      file1.close()

      !./marabou_DnC_InternalNodes.elf ./ACASXU_run2a_1_1_batch_2000.nnet ./property.txt --summary-file=summary1.txt

      print("SUMMARY:")
      f = open('summary1.txt', 'r')
      file_contents = f.read()
      print (file_contents)
      f.close()
      if (file_contents.find('UNSAT') == -1):
         notCov.append(strInternal)

    return notCov
  #f.close()


In [0]:
print("layer:", LAYER)
#",label:", LABEL)
invariants = get_all_invariants(basic_estimator)
#describe_invariants_all_labels(invariants,prev_lay,curr_lay)

pr = 10
label = 0
pr_minIn = [0.268978,0.111408,-0.5, 0.227273,0.0]
pr_maxIn = [0.679858,0.499999,-0.49841,0.5,0.5]

print("CHECK PROPERTY:", str(pr) )
print("REGION A:")
print(pr_minIn)
print(pr_maxIn)
print("LABEL B:", str(label))


property_chk(pr,label,invariants,pr_minIn,pr_maxIn)

#pr = 10
#label = 0
#layer_tens = [h_fc1,h_fc2,h_fc3,h_fc4,h_fc5,h_fc6]
#for lay in range(0,1):
#  print("layer:", lay)
#  train_suffixes = fingerprint_suffix(acas_train,layer_tens[lay])
#  print("Suffixes computed for all training data")
#  train_predictions = np.argmin(get_prediction(acas_train), axis=1)
#  basic_estimator = tree.DecisionTreeClassifier()
#  basic_estimator.fit(train_suffixes, train_predictions)
#  invariants = get_all_invariants(basic_estimator)
 






layer: 5


100%|██████████| 9117/9117 [00:00<00:00, 927152.30it/s]

Obtained all paths
CHECK PROPERTY: 10
REGION A:
[0.268978, 0.111408, -0.5, 0.227273, 0.0]
[0.679858, 0.499999, -0.49841, 0.5, 0.5]
LABEL B: 0



100%|██████████| 2/2 [00:00<00:00, 260.27it/s]

Property:10 ,A => LABEL0
INPUTS WITHIN A: 195
0 836
INVARIANT > 100: [41, 21, 2, 40, 43, 9, 46, 22, 26, 27, 34, 3, 8, 42, 4, 15, 20, 23, 37, 45] [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SUPPORT > 100: 109147 , 109147 , A SUPPORT: 195
COVERED:
INP MIN,MAX covered:
[0.301769, 0.15, -0.5, 0.357182, 0.0]
[0.679858, 0.474998, -0.5, 0.5, 0.5]
MIN, MAX LAYER INPS: 195
INP MIN,MAX PREV LAYER:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.7928456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0954626, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.986864, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.041381, 0.0, 0.0, 0.0]
[0.48795617, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.7465641, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6651177, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17363247, 0.0, 9.182408, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.1081314, 1.0279957, 2.1200955, 0.0]
COMMON SIGNATURE:
i

Network: ./ACASXU_run2a_1_2_batch_fc5OP.nnet
Number of layers: 3. Input layer size: 50. Output layer size: 5. Number of ReLUs: 50
Total number of variables: 155
Property: ./property.txt

Engine::processInputQuery: Input query (before preprocessing): 56 equations, 155 variables
Engine::processInputQuery: Input query (after preprocessing): 106 equations, 125 variables

Input bounds:
	x0: [  0.0000,   0.4880] 
	x1: [  0.0000,   0.0000] [FIXED]
	x2: [  0.0000,   0.0000] [FIXED]
	x3: [  0.0000,   0.0000] [FIXED]
	x4: [  0.0000,   0.0000] [FIXED]
	x5: [  0.0000,   0.0000] [FIXED]
	x6: [  0.0000,   0.0000] [FIXED]
	x7: [  0.0000,   0.0000] [FIXED]
	x8: [  0.0000,   0.0000] [FIXED]
	x9: [  0.0000,   0.0000] [FIXED]
	x10: [  1.7928,   3.7466] 
	x11: [  0.0000,   0.0000] [FIXED]
	x12: [  0.0000,   0.0000] [FIXED]
	x13: [  0.0000,   0.0000] [FIXED]
	x14: [  0.0000,   0.0000] [FIXED]
	x15: [  0.0000,   0.0000] [FIXED]
	x16: [  0.0000,   0.0000] [FIXED]
	x17: [  0.0000,   0.0000] [FIXED]
	x18: [  0